In [379]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
df=pd.read_excel("data_manage/question1/initial_data/nipt_data.xlsx", sheet_name="男胎检测数据")
df = df[df['Y染色体浓度'].notna()].copy()

def convert_ga_to_decimal(ga_str):
    """将“Xw+Y”格式转换为小数周（如11w+6 = 11 + 6/7 ≈ 11.86周）"""
    if pd.isna(ga_str):
        return np.nan
    # 拆分“周”和“天”部分，适配实际数据格式
    if 'w+' in str(ga_str):
        w_part, d_part = str(ga_str).split('w+')
    else:
        return np.nan  # 排除格式异常的孕周（如纯数字）
    try:
        return int(w_part) + int(d_part) / 7
    except (ValueError, IndexError):
        return np.nan

# 转换孕周并筛选10-25周样本
df['小数孕周'] = df['检测孕周'].apply(convert_ga_to_decimal)
df = df[(df['小数孕周'] >= 10) & (df['小数孕周'] <= 25)]

# 4. 测序质量筛选（C题.pdf附录1：GC含量正常范围40%~60%，对应真实列名“GC含量”（列16））
# 注意：数据中GC含量可能以小数形式呈现（如0.377069=37.7%），需先转换为百分比
df['GC含量_百分比'] = df['GC含量'] * 100
df = df[(df['GC含量_百分比'] >= 40) & (df['GC含量_百分比'] <= 60)]

# 5. 剔除Y染色体浓度异常值（C题.pdf4：男胎Y浓度达标值≥4%，数据中为小数（如0.025936=2.59%），合理范围0~15%）
df['Y染色体浓度_百分比'] = df['Y染色体浓度'] * 100
df = df[(df['Y染色体浓度_百分比'] >= 0) & (df['Y染色体浓度_百分比'] <= 15)]

# 6. 输出筛选结果（验证数据有效性）
print("=== 数据筛选结果 ===")
print(f"原始数据总样本数：{len(df)}")
print(f"男胎样本数（Y染色体浓度非空）：{len(df[df['Y染色体浓度'].notna()])}")
print(f"10-25周+正常GC的有效男胎样本数：{len(df)}")

# 7. 查看核心变量描述统计（贴合C题.pdf问题1的分析对象）
print("\n=== 核心变量描述统计（有效样本） ===")
core_vars = df[['Y染色体浓度_百分比', '小数孕周', '孕妇BMI']].rename(columns={
    'Y染色体浓度_百分比': 'Y染色体浓度(%)',
    '小数孕周': '孕周（小数）',
    '孕妇BMI': 'BMI'
})
print(core_vars.describe())

# 标记同一孕妇的多次检测（C题.pdf6：存在多次采血检测，对应真实列名“孕妇代码”（列2））
df['检测次数'] = df.groupby('孕妇代码')['检测日期'].cumcount() + 1  # 按检测日期排序标记次数

# 查看重复检测示例（以孕妇A001为例，匹配你提供的前3行数据）
print("\n=== 同一孕妇多次检测示例（孕妇A001） ===")
a001_data = df[df['孕妇代码'] == 'A001'][
    ['孕妇代码', '检测次数', '检测孕周', '小数孕周', 'Y染色体浓度_百分比', '孕妇BMI']
].sort_values('检测次数')

print(a001_data)
df.to_csv("data_manage/question1/initial_data/nipt_data_processed.csv", index=False)

=== 数据筛选结果 ===
原始数据总样本数：519
男胎样本数（Y染色体浓度非空）：519
10-25周+正常GC的有效男胎样本数：519

=== 核心变量描述统计（有效样本） ===
        Y染色体浓度(%)      孕周（小数）         BMI
count  519.000000  519.000000  519.000000
mean     7.574064   16.120286   32.304640
std      2.916116    3.642887    2.747084
min      1.000389   11.000000   26.619343
25%      5.219353   13.142857   30.370302
50%      7.630164   15.571429   32.017138
75%      9.839118   17.857143   33.920238
max     14.825457   24.857143   46.875000

=== 同一孕妇多次检测示例（孕妇A001） ===
Empty DataFrame
Columns: [孕妇代码, 检测次数, 检测孕周, 小数孕周, Y染色体浓度_百分比, 孕妇BMI]
Index: []


In [381]:
Y_data_whole=df['Y染色体浓度']
wk_data_whole=df['小数孕周']
BMI_data_whole=df['孕妇BMI']
patient_id=df['孕妇代码']

print(f"bmi_values length: {len(Y_data_whole)}")
print(f"event_times length: {len(wk_data_whole)}")
print(f"event_observed length: {len(BMI_data_whole)}")
data=pd.DataFrame({'孕妇代码':patient_id, '孕周_小数':wk_data_whole, 'Y染色体浓度_百分比':Y_data_whole, '孕妇BMI':BMI_data_whole})

bmi_values length: 519
event_times length: 519
event_observed length: 519


In [382]:
import numpy as np
import jenkspy
from lifelines import KaplanMeierFitter

bmi_values = BMI_data_whole
covariates = Y_data_whole
event_times = wk_data_whole
event_observed = (Y_data_whole >= 0.04).astype(int)
initial_times = [20,20,20,20]  # 每组初始时间点

# -----------------------
# 1. 分组（Jenks）
# -----------------------
def initial_grouping(bmi_values, n_classes):
    breaks = jenkspy.jenks_breaks(bmi_values, n_classes)
    labels = np.digitize(bmi_values, bins=breaks, right=True) - 1
    # 防止越界
    labels = np.clip(labels, 0, n_classes - 1)
    return labels, breaks

# -----------------------
# 2. Kaplan-Meier 拟合
# -----------------------
def fit_and_evaluate(event_times, labels, event_observed):
    kmf_dict = {}
    for label in np.unique(labels):
        mask = (labels == label)
        kmf = KaplanMeierFitter()
        kmf.fit(event_times[mask], event_observed=event_observed[mask])
        kmf_dict[label] = kmf
    return kmf_dict

def get_survival_probability(kmf_dict, label, time_point):
    S = kmf_dict[label].predict(time_point)
    if np.isnan(S):
        return 0.0  # 超出范围时生存率视为0
    return float(S)

# -----------------------
# 3. 风险函数（单组单时间点）
# -----------------------
def expected_risk_func(time, label, kmf_dict):
    c_early, c_mid, c_late = 0.5, 0.8, 1.0
    t_c_early, t_c_mid = 12, 27
    S_T = get_survival_probability(kmf_dict, label, time)
    if time <= t_c_early:
        return c_early * (1 - S_T)
    elif t_c_early < time < t_c_mid:
        return c_mid * (1 - S_T)
    else:
        return c_late * (1 - S_T)

def compute_single_gradient(time, label, kmf_dict, epsilon=1):
    perturbed_plus = expected_risk_func(time + epsilon, label, kmf_dict)
    perturbed_minus = expected_risk_func(time - epsilon, label, kmf_dict)
    grad = (perturbed_plus - perturbed_minus) / (2 * epsilon)
    if not np.isfinite(grad):
        grad = 0.0
    return grad

# -----------------------
# 4. 梯度下降优化（每组独立优化时间）
# -----------------------
def optimize_group(label, time, kmf_dict, learning_rate=0.01, max_iter=1000, tol=1, max_step=1.0, beta=0.9, epsilon=1e-8):
    max_time = kmf_dict[label].timeline[-1]
    s = 0.0  # RMSProp state
    for iteration in range(max_iter):
        grad = compute_single_gradient(time, label, kmf_dict)
        # RMSProp update
        s = beta * s + (1 - beta) * grad**2
        step = learning_rate * grad / (np.sqrt(s) + epsilon)
        step = np.clip(step, -max_step, max_step)
        time -= step
        # Clip time to allowable range
        time = np.clip(time, 0.0, max_time)
        if abs(grad) < tol:
            print(f"Group {label} converged at iteration {iteration}")
            break
        # Print debug information during optimization
        if iteration % 100 == 0 or abs(grad) < tol:  # You can adjust this interval
            risk = expected_risk_func(time, label, kmf_dict)
            print(f"Iteration {iteration}, Group {label}: Time = {time:.4f}, Risk = {risk:.6f}, Grad = {grad:.6f}")

    print(f"Group {label}: Optimal time = {time:.4f}")
    return time

def rmsprop_optimize(bmi_values, event_times, event_observed, initial_times, n_classes=4):
    labels, breaks = initial_grouping(bmi_values, n_classes)
    kmf_dict = fit_and_evaluate(event_times, labels, event_observed)
    optimal_times = []

    for label in range(n_classes):
        print(f"Optimizing group {label}...")
        optimal_time = optimize_group(label, initial_times[label], kmf_dict)
        optimal_times.append(optimal_time)

    print(f"All groups optimized. Optimal times: {optimal_times}")
    return optimal_times

# Call the rmsprop_optimize function
optimal_times = rmsprop_optimize(bmi_values=bmi_values, event_times=event_times, event_observed=event_observed,
                 initial_times=initial_times)

Optimizing group 0...
Group 0 converged at iteration 0
Group 0: Optimal time = 19.9684
Optimizing group 1...
Group 1 converged at iteration 0
Group 1: Optimal time = 19.9684
Optimizing group 2...
Group 2 converged at iteration 0
Group 2: Optimal time = 19.9684
Optimizing group 3...
Group 3 converged at iteration 0
Group 3: Optimal time = 19.9684
All groups optimized. Optimal times: [np.float64(19.96837724100106), np.float64(19.968377243895294), np.float64(19.968377248288334), np.float64(19.96837723353535)]


In [383]:
from lifelines import KaplanMeierFitter
import numpy as np

# 假设你的事件时间和事件观测状态
event_times = np.array([5, 10, 15, 20, 25])
events_observed = np.array([1, 1, 0, 1, 0])

# 初始化和拟合KaplanMeierFitter
kmf = KaplanMeierFitter()
kmf.fit(event_times, event_observed=events_observed)

time = 12
survival_probability = kmf.predict(time)
print(f"Survival probability at time {time}: {survival_probability}")

Survival probability at time 12: 0.6000000000000001
